In [5]:
import tensorflow as tf
import pandas as pd
import pathlib
import tqdm
import os
import cv2
import re
import csv

In [6]:
pathVid = "D:\My Documents\GIT Projects\TDL-Hackathon_Day-1\input\Training Dataset_Videos"
pathCsv = "D:\My Documents\GIT Projects\TDL-Hackathon_Day-1\input\Training Dataset_csv"
vidAddr = []
csvAddr = []

for root, dirs, files in os.walk(pathVid):
       print(root)
       for _file in files:
           filename = os.fsdecode(_file)
           if filename.endswith(".mp4"):
                 fileAddr = root+"\\"+_file
                 vidAddr.append(fileAddr)

for root, dirs, files in os.walk(pathCsv):
       print(root)
       for _file in files:
           filename = os.fsdecode(_file)
           if filename.endswith(".csv"):
                 fileAddr = root+"\\"+_file
                 csvAddr.append(fileAddr)

print(vidAddr)

print(csvAddr)


D:\My Documents\GIT Projects\TDL-Hackathon_Day-1\input\Training Dataset_Videos
D:\My Documents\GIT Projects\TDL-Hackathon_Day-1\input\Training Dataset_csv
['D:\\My Documents\\GIT Projects\\TDL-Hackathon_Day-1\\input\\Training Dataset_Videos\\V_AIRPLANE_001.mp4', 'D:\\My Documents\\GIT Projects\\TDL-Hackathon_Day-1\\input\\Training Dataset_Videos\\V_AIRPLANE_002.mp4', 'D:\\My Documents\\GIT Projects\\TDL-Hackathon_Day-1\\input\\Training Dataset_Videos\\V_AIRPLANE_003.mp4', 'D:\\My Documents\\GIT Projects\\TDL-Hackathon_Day-1\\input\\Training Dataset_Videos\\V_AIRPLANE_004.mp4', 'D:\\My Documents\\GIT Projects\\TDL-Hackathon_Day-1\\input\\Training Dataset_Videos\\V_AIRPLANE_005.mp4', 'D:\\My Documents\\GIT Projects\\TDL-Hackathon_Day-1\\input\\Training Dataset_Videos\\V_AIRPLANE_006.mp4', 'D:\\My Documents\\GIT Projects\\TDL-Hackathon_Day-1\\input\\Training Dataset_Videos\\V_AIRPLANE_007.mp4', 'D:\\My Documents\\GIT Projects\\TDL-Hackathon_Day-1\\input\\Training Dataset_Videos\\V_AIRPLAN

In [7]:
for i in range(len(vidAddr)):
    curr_df = pd.read_csv(csvAddr[i],header=None).dropna()
    cap = cv2.VideoCapture(vidAddr[i])
    count = 0
    for index, row in curr_df.iterrows():
        if(row[0][0]=="'"):
            ts = float(row[0][1:-5])
        else:
            ts = float(row[0][:-4])
        cap.set(cv2.CAP_PROP_POS_MSEC,ts)
        ret,frame = cap.read()
        cv2.imwrite(f"Output/TrainingImages/{vidAddr[i][81:-4]}_{count}.jpg",frame)
        count+=1
    cap.release()

In [4]:
folder_path = 'Output/TrainingImages/'
files = [f for f in os.listdir(folder_path)]

In [5]:
df = pd.DataFrame(files, columns=['Name/path'])
df['Name/path'] = df['Name/path'].apply(lambda x: os.path.join(folder_path, x))

In [6]:
def get_label(file_name):
    file_name_no_ext = os.path.splitext(file_name)[0]
    label = re.sub(r'\d+', '', file_name_no_ext)
    return label[:-2]

df['Label'] = df['Name/path'].apply(lambda x: get_label(os.path.basename(x)))
df

,Name/path,Label
0,Output/TrainingImages/AIRPLANE_001_0.jpg,AIRPLANE
1,Output/TrainingImages/AIRPLANE_001_1.jpg,AIRPLANE
2,Output/TrainingImages/AIRPLANE_001_10.jpg,AIRPLANE
3,Output/TrainingImages/AIRPLANE_001_100.jpg,AIRPLANE
4,Output/TrainingImages/AIRPLANE_001_101.jpg,AIRPLANE
...,...,...
87360,Output/TrainingImages/HELICOPTER_061_95.jpg,HELICOPTER
87361,Output/TrainingImages/HELICOPTER_061_96.jpg,HELICOPTER
87362,Output/TrainingImages/HELICOPTER_061_97.jpg,HELICOPTER
87363,Output/TrainingImages/HELICOPTER_061_98.jpg,HELICOPTER


In [7]:
dataframe = pd.DataFrame()
for i in range(len(csvAddr)):
    current_df = pd.read_csv(csvAddr[i],names=['Timestamp','Airplane','Bird','Drone','Helicopter'])
    if(dataframe.empty):
        dataframe =current_df
    else:
        dataframe = pd.concat([dataframe,current_df])


dataframe = dataframe.replace({"[]":""})
dataframe["bounding_box"] = dataframe['Airplane'] + dataframe['Bird'] + dataframe['Drone'] + dataframe['Helicopter']
dataframe = dataframe.drop(["Timestamp","Airplane","Bird","Drone","Helicopter"],axis=1)
dataframe
df.insert(2,"Bounding_box",dataframe)

,bounding_box
0,"[313,205,36,24]"
1,"[313.448181152344,204.424362182617,36.06567382..."
2,"[311.489379882813,204.399459838867,36.05471801..."
3,"[309.946655273438,203.389114379883,36.08734130..."
4,"[308.959167480469,203.633331298828,36.10455322..."
...,...
302,"[18.1111147058835,331.555555411765,26.77777835..."
303,"[15.3333360294126,331.666666558824,26.33333376..."
304,"[12.5555573529418,331.777777705882,25.88888917..."
305,"[9.77777867647088,331.888888852941,25.44444458..."


In [147]:
df

,Name/path,Label,bounding_box
0,Output/TrainingImages/AIRPLANE_001_0.jpg,AIRPLANE,"[313,205,36,24]"
0,Output/TrainingImages/AIRPLANE_001_0.jpg,AIRPLANE,"[342,151,49,37]"
0,Output/TrainingImages/AIRPLANE_001_0.jpg,AIRPLANE,"[382,234,68,33]"
0,Output/TrainingImages/AIRPLANE_001_0.jpg,AIRPLANE,"[291.611175537109,261.708496093750,28.30804443..."
0,Output/TrainingImages/AIRPLANE_001_0.jpg,AIRPLANE,"[285,80,45.3704245973645,41.3162518301610]"
...,...,...,...
87360,Output/TrainingImages/HELICOPTER_061_95.jpg,HELICOPTER,NaN
87361,Output/TrainingImages/HELICOPTER_061_96.jpg,HELICOPTER,NaN
87362,Output/TrainingImages/HELICOPTER_061_97.jpg,HELICOPTER,NaN
87363,Output/TrainingImages/HELICOPTER_061_98.jpg,HELICOPTER,NaN
